In [58]:
#
# pokersim.py - Runs a Monte Carlo simulation of two Texas Hold'em hands
#               with user-specified (or random) community cards
#
# Work to be done:
# Add exhaustive search?
# Add input checking for user input
#   No duplicate cards or invalid input
# Add support for random rank, defined suit (and vice versa)
# Add support for random 'hand2' cards
# Add test cases
#
import argparse
import random
import numpy as np


def readable_hand(cards):
    #
    # Returns a readable version of a set of cards
    #
    card_rank = {0: "2", 1: "3", 2: "4", 3: "5", 4: "6", 5: "7", 6: "8",
                 7: "9", 8: "T", 9: "J", 10: "Q", 11: "K", 12: "A", -1: "X"}
    card_suit = {0: "c", 1: "d", 2: "h", 3: "s", -1: "x"}
    return_string = ""
    for i in cards:
        return_string += card_rank[i[0]] + card_suit[i[1]]
    return return_string


def hand_copy(cards):
    #
    # Returns copy of hand (replaces deepcopy with 20x speed improvement)
    #
    results = []
    for i in cards:
        results.append(i)
    return results


def legal_hand(cards):
    #
    # Returns True if hand is legal
    # Returns False if hand is illegal
    #   Case 1: two or more of same card
    #   Case 2: random card
    #
    for i in cards:
        if cards.count(i) > 1 or cards == [-1, -1]:
            return False
    return True


def valid_card(card):
    #
    # Returns True if card is a valid card in text format (rank in (A-2),
    #  suit in (c, d, h, s) or wildcard (Xx)
    # Returns False if card is invalid
    #
    if card[0] in ("X", "x", "A", "a", "K", "k", "Q", "q", "J", "j",
                   "T", "t", "9", "8", "7", "6", "5", "4", "3", "2") \
            and card[1] in ("x", "X", "c", "C", "d", "D", "h", "H", "s", "S"):
        return True
    else:
        return False


def hand_to_numeric(cards):
    #
    # Converts alphanumeric hand to numeric values for easier comparisons
    # Also sorts cards based on rank
    #
    card_rank = {"2": 0, "3": 1, "4": 2, "5": 3, "6": 4, "7": 5, "8": 6,
                 "9": 7, "T": 8, "J": 9, "Q": 10, "K": 11, "A": 12, "X": -1,
                         "t": 8, "j": 9, "q": 10, "k": 11, "a": 12, "x": -1}
    card_suit = {"c": 0, "C": 0, "d": 1, "D": 1, "h": 2, "H": 2,
                 "s": 3, "S": 3, "x": -1, "X": -1}
    result = []
    for i in range(len(cards) // 2 + len(cards) % 2):
        result.append([card_rank[cards[i * 2]], card_suit[cards[i * 2 + 1]]])
    result.sort()
    result.reverse()
    return result


def check_flush(hand):
    #
    # Returns True if hand is a Flush, otherwise returns False
    #
    hand_suit = [hand[0][1], hand[1][1], hand[2][1], hand[3][1], hand[4][1]]
    for i in range(4):
        if hand_suit.count(i) == 5:
            return True
    return False


def check_straight(hand):
    # Return True if hand is a Straight, otherwise returns False
    if hand[0][0] == (hand[1][0] + 1) == (hand[2][0] + 2) == (hand[3][0] + 3)\
            == (hand[4][0] + 4):
        return True
    elif (hand[0][0] == 12) and (hand[1][0] == 3) and (hand[2][0] == 2)\
            and (hand[3][0] == 1) and (hand[4][0] == 0):
        return True
    return False


def check_straightflush(hand):
    # Return True if hand is a Straight Flush, otherwise returns False
    if check_flush(hand) and check_straight(hand):
        return True
    return False


def check_fourofakind(hand):
    # Return True if hand is Four-of-a-Kind, otherwise returns False
    # Also returns rank of four of a kind card and rank of fifth card
    # (garbage value if no four of a kind)
    hand_rank = [hand[0][0], hand[1][0], hand[2][0], hand[3][0], hand[4][0]]
    for quad_card in range(13):
        if hand_rank.count(quad_card) == 4:
            for kicker in range(13):
                if hand_rank.count(kicker) == 1:
                    return True, quad_card, kicker
    return False, 13, 13


def check_fullhouse(hand):
    # Return True if hand is a Full House, otherwise returns False
    # Also returns rank of three of a kind card and two of a kind card
    # (garbage values if no full house)
    hand_rank = [hand[0][0], hand[1][0], hand[2][0], hand[3][0], hand[4][0]]
    for trip_card in range(13):
        if hand_rank.count(trip_card) == 3:
            for pair_card in range(13):
                if hand_rank.count(pair_card) == 2:
                    return True, trip_card, pair_card
    return False, 13, 13


def check_threeofakind(hand):
    # Return True if hand is Three-of-a-Kind, otherwise returns False
    # Also returns rank of three of a kind card and remaining two cards
    # (garbage values if no three of a kind)
    hand_rank = [hand[0][0], hand[1][0], hand[2][0], hand[3][0], hand[4][0]]
    for trip_card in range(13):
        if hand_rank.count(trip_card) == 3:
            for n in range(13):
                if hand_rank.count(n) == 1:
                    for m in range(n+1, 13):
                        if hand_rank.count(m) == 1:
                            return True, trip_card, [m, n]
    return False, 13, [13, 13]


def check_twopair(hand):
    # Return True if hand is Two Pair, otherwise returns False
    # Also returns ranks of paired cards and remaining card
    # (garbage values if no two pair)
    hand_rank = [hand[0][0], hand[1][0], hand[2][0], hand[3][0], hand[4][0]]
    for low_pair_card in range(13):
        if hand_rank.count(low_pair_card) == 2:
            for high_pair_card in range(low_pair_card + 1, 13):
                if hand_rank.count(high_pair_card) == 2:
                    for kicker in range(13):
                        if hand_rank.count(kicker) == 1:
                            return True, [high_pair_card, low_pair_card], \
                                kicker
    return False, [13, 13], 13


def check_onepair(hand):
    # Return True if hand is One Pair, otherwise returns False
    # Also returns ranks of paired cards and remaining three cards
    # (garbage values if no pair)
    hand_rank = [hand[0][0], hand[1][0], hand[2][0], hand[3][0], hand[4][0]]
    for pair_card in range(13):
        if hand_rank.count(pair_card) == 2:
            for kicker1 in range(13):
                if hand_rank.count(kicker1) == 1:
                    for kicker2 in range(kicker1 + 1, 13):
                        if hand_rank.count(kicker2) == 1:
                            for kicker3 in range(kicker2 + 1, 13):
                                if hand_rank.count(kicker3) == 1:
                                    return True, pair_card, \
                                        [kicker3, kicker2, kicker1]
    return False, 13, [13, 13, 13]


def highest_card(hand1, hand2):
    # Return 0 if hand1 is higher
    # Return 1 if hand2 is higher
    # Return 2 if equal
    hand1_rank = \
        [hand1[0][0], hand1[1][0], hand1[2][0], hand1[3][0], hand1[4][0]]
    hand2_rank = \
        [hand2[0][0], hand2[1][0], hand2[2][0], hand2[3][0], hand2[4][0]]
    #
    # Compare
    #
    if hand1_rank > hand2_rank:
        return 0
    elif hand1_rank < hand2_rank:
        return 1
    return 2


def highest_card_straight(hand1, hand2):
    # Return 0 if hand1 is higher
    # Return 1 if hand2 is higher
    # Return 2 if equal
    #
    # Compare second card first (to account for Ace low straights)
    # if equal, we could have Ace low straight, so compare first card.
    # If first card is Ace, that is the lower straight
    #
    if hand1[1][0] > hand2[1][0]:
        return 0
    elif hand1[1][0] < hand2[1][0]:
        return 1
    elif hand1[0][0] > hand2[0][0]:
        return 1
    elif hand1[0][0] < hand2[0][0]:
        return 0
    return 2


def compare_hands(hand1, hand2):
    #
    # Compare two hands
    # Return 0 if hand1 is better
    # Return 1 if hand2 is better
    # Return 2 if equal
    #
    result1 = []
    result2 = []
    #
    # Check for straight flush
    #
    if check_straightflush(hand1):
        if check_straightflush(hand2):
            return(highest_card_straight(hand1, hand2))
        else:
            return 0
    elif check_straightflush(hand2):
            return 1
    #
    # Check for four of a kind
    #
    result1 = check_fourofakind(hand1)
    result2 = check_fourofakind(hand2)
    if result1[0] == 1:
        if result2[0] == 1:
            if result1[1] > result2[1]:
                return 0
            elif result1[1] < result2[1]:
                return 1
            elif result1[2] > result2[2]:
                return 0
            elif result1[2] < result2[2]:
                return 1
            else:
                return 2
        else:
            return 0
    elif result2[0] == 1:
        return 1
    #
    # Check for full house
    #
    result1 = check_fullhouse(hand1)
    result2 = check_fullhouse(hand2)
    if result1[0] == 1:
        if result2[0] == 1:
            if result1[1] > result2[1]:
                return 0
            elif result1[1] < result2[1]:
                return 1
            elif result1[2] > result2[2]:
                return 0
            elif result1[2] < result2[2]:
                return 1
            else:
                return 2
        else:
            return 0
    elif result2[0] == 1:
        return 1
    #
    # Check for flush
    #
    if check_flush(hand1):
        if check_flush(hand2):
            return(highest_card(hand1, hand2))
        else:
            return 0
    elif check_flush(hand2):
        return 1
    #
    # Check for straight
    #
    if check_straight(hand1):
        if check_straight(hand2):
            temp = highest_card_straight(hand1, hand2)
            return temp
        else:
            return 0
    elif check_straight(hand2):
        return 1
    #
    # Check for three of a kind
    #
    result1 = check_threeofakind(hand1)
    result2 = check_threeofakind(hand2)
    if result1[0] == 1:
        if result2[0] == 1:
            if result1[1] > result2[1]:
                return 0
            elif result1[1] < result2[1]:
                return 1
            elif result1[2] > result2[2]:
                return 0
            elif result1[2] < result2[2]:
                return 1
            else:
                return 2
        else:
            return 0
    elif result2[0] == 1:
        return 1
    #
    # Check for two pair
    #
    result1 = check_twopair(hand1)
    result2 = check_twopair(hand2)
    if result1[0] == 1:
        if result2[0] == 1:
            if result1[1] > result2[1]:
                return 0
            elif result1[1] < result2[1]:
                return 1
            elif result1[2] > result2[2]:
                return 0
            elif result1[2] < result2[2]:
                return 1
            else:
                return 2
        else:
            return 0
    elif result2[0] == 1:
        return 1
    #
    # Check for one pair
    #
    result1 = check_onepair(hand1)
    result2 = check_onepair(hand2)
    if result1[0] == 1:
        if result2[0] == 1:
            if result1[1] > result2[1]:
                return 0
            elif result1[1] < result2[1]:
                return 1
            elif result1[2] > result2[2]:
                return 0
            elif result1[2] < result2[2]:
                return 1
            else:
                return 2
        else:
            return 0
    elif result2[0] == 1:
        return 1
    return (highest_card(hand1, hand2))


def best_five(hand, community):
    #
    # Takes hand and community cards in numeric form
    # Returns best five cards
    #
    currentbest = hand_copy(community)
    currentbest.sort()
    currentbest.reverse()
    #
    # Compare current best to five cards including only one player card
    #
    for m in range(2):
        for n in range(5):
            comparehand = hand_copy(community)
            comparehand[n] = hand[m]
            comparehand.sort()
            comparehand.reverse()
            if compare_hands(currentbest, comparehand) == 1:
                currentbest = hand_copy(comparehand)
    #
    # Compare current best to five cards including both player cards
    #
    for m in range(5):
        for n in range(m + 1, 5):
            comparehand = hand_copy(community)
            comparehand[m] = hand[0]
            comparehand[n] = hand[1]
            comparehand.sort()
            comparehand.reverse()
            if compare_hands(currentbest, comparehand) == 1:
                currentbest = hand_copy(comparehand)
    return currentbest



   
iterations = 1000

community = "XxXxXxXxXx"


hand1 = "AsAh"
handnum1 = hand_to_numeric(hand1)
hand2 = "KsKh"
handnum2 = hand_to_numeric(hand2)

    # Initialize counters
totals = [0, 0, 0]
    # Monte Carlo Simulation
start = time.time()
for _ in range(iterations):
    community_original = hand_to_numeric(community)
    community_temp = community_original[:]
    while not legal_hand(handnum1 + handnum2 + community_temp):
        community_temp = community_original[:]
        for i in range(len(community_temp)):
            if community_temp[i][0] == -1:
                community_temp[i] = [random.randint(0, 12),
                                     random.randint(0, 3)]
    best_hand1 = best_five(handnum1, community_temp)
    best_hand2 = best_five(handnum2, community_temp)
    totals[compare_hands(best_hand1, best_hand2)] += 1
    # Print results
end = time.time()
elapsed = end - start
print ("Total Hands: %i" % (iterations))
print ("Hand1: %i Hand2: %i Ties: %i" % (totals[0], totals[1], totals[2]))
print ("Hand1: %.2f%% Hand2: %.2f%% Ties: %.2f%%" % \
        (100 * round((totals[0] / (iterations + 0.0)), 4),
         100 * round((totals[1] / (iterations + 0.0)), 4),
         100 * round((totals[2] / (iterations + 0.0)), 4)))
print(elapsed)


Total Hands: 1000
Hand1: 836 Hand2: 160 Ties: 4
Hand1: 83.60% Hand2: 16.00% Ties: 0.40%
1.6583752632141113


In [59]:
#N - number of times to simulate the round
#n_players - number of other players in game
#starting_hand - your 2 starting cards you were dealt
def monteCarloPoker(N, n_players, starting_hand):
    win_probability = 0
    lose_probability = 0
    tie_probability = 0
    for i in range(N):
        #create deck
        deck = np.array([])
        for i in range(52):
            deck = np.append(deck, i)
        deck = deck.astype(int)

        #create starting hand
        start_hand = np.array([])
        for card in starting_hand:
            card_value = cardNameToInt(card)
            start_hand = np.append(start_hand, card_value)
        start_hand = start_hand.astype(int)

        #remove starting_hand from deck
        index1 = np.argwhere(deck == start_hand[0])
        deck = np.delete(deck, index1)

        index2 = np.argwhere(deck == start_hand[1])
        deck = np.delete(deck, index2)

        #generate hands for game
        dealer_tup = generateDealerCards(deck)
        dealer_cards = dealer_tup[0]
        deck = dealer_tup[1]
    
        player_tup = generatePlayersCards(n_players, deck)
        players_cards = player_tup[0]
        deck = dealer_tup[1]

        #determine winner
        result = whoWin(start_hand, players_cards, dealer_cards)

        #determine probabilities
        if result == 0:
            win_probability += 1/N
        if result == 1:
            lose_probability += 1/N
        if result == 2:
            tie_probability += 1/N

    return [win_probability, lose_probability, tie_probability]

In [60]:
#convert card name from string such as 5c
#to a value from 0-51 ordered such as
#2c,2d,2h,2s,3c,3d,3h,3s,... 
def cardNameToInt(name):
    suits = ['c','d','h','s']
    face_cards = {'T':10,'J':11,'Q':12,'K':13,'A':14}
   
    suit_value = suits.index(name[1])
    card_num = name[0]
    
    if card_num in face_cards.keys():
        card_num = face_cards[card_num]
    else:
        card_num = int(card_num)
    
    return suit_value + 4 * (card_num-2)

#generate starting hands for all the other players in the game
#returns an array containing arrays of each player's starting cards
def generatePlayersCards(n_players,available_deck):
    updated_card_deck = available_deck
    np.random.shuffle(updated_card_deck)
    players_cards = np.empty((0,2))
    for i in range(n_players):
        temp = np.array([[updated_card_deck[0],updated_card_deck[1]]])
        players_cards = np.append(players_cards, temp, axis =0)
        updated_card_deck = np.delete(updated_card_deck, 0)
        updated_card_deck = np.delete(updated_card_deck, 0)
    players_cards = players_cards.astype(int)
    return(players_cards,updated_card_deck)

#generate the cards for the flop, turn, and river
def generateDealerCards(available_deck):
    updated_card_deck = available_deck
    np.random.shuffle(updated_card_deck)
    dealer_cards = np.array([])
    for i in range(5):
        dealer_cards = np.append(dealer_cards, updated_card_deck[0])
        updated_card_deck = np.delete(updated_card_deck, 0)
    dealer_cards = dealer_cards.astype(int)
    return(dealer_cards,updated_card_deck)

In [61]:
#lists of cards categorized by their number/face
twos = [i for i in range (0,4)]
threes = [i for i in range (4,8)]
fours = [i for i in range(8,12)]
fives = [i for i in range(12,16)]
sixes = [i for i in range(16,20)]
sevens = [i for i in range(20,24)]
eights = [i for i in range(24,28)]
nines = [i for i in range(28,32)]
tens = [i for i in range(32,36)]
jacks = [i for i in range(36,40)]
queens = [i for i in range(40,44)]
kings = [i for i in range(44,48)]
aces = [i for i in range(48,52)]

card_nums = [twos,threes,fours,fives,sixes,sevens,eights,nines,tens,jacks,queens,kings,aces]

#convert cards from 0-51 value to thier number/face value
def cardIntsToCardNums(hand):
    #list to hold number/face values of cards
    hand_nums = []

    #convert cards from 0-51 value to thier number/face value
    for i in range(len(hand)):
        for j in range(len(card_nums)):
            if hand[i] in card_nums[j]:
                hand_nums.append(j+2)
    
    return hand_nums

def isFlush(hand):
    #lists with all the cards of each suit
    clubs = [i for i in range(0,52,4)]
    diamonds = [i for i in range(1,52,4)]
    hearts = [i for i in range(2,52,4)]
    spades = [i for i in range(3,52,4)]

    suits = [clubs,diamonds,hearts,spades]

    #check both players' hands for flushes
    for suit in suits:
        if len(np.intersect1d(hand, suit)) >= 5:
            return True
    return False
    
def isStraight(hand):
    #convert list of card values to numbers/faces
    player_nums = cardIntsToCardNums(hand)
    
    #check if the card numbers/faces are consecutive to determine if hand is a straight
    player_nums = np.sort(player_nums)
    straight = np.array(list(range(np.min(player_nums), np.max(player_nums)+1)))
    if np.array_equal(player_nums, straight):
        return True
    else: 
        return False

def isStraightFlush(hand):
    if isFlush(hand) and isStraight(hand):
        return True
    else:
        return False

def isRoyalFlush(hand):
    #checks if hand is a straight flush with lowest card being a ten
    if isStraightFlush(hand) and (np.min(hand) in range(32,36)):
        return True
    else:
        return False

def isFourOfAKind(hand):
    for card_num in card_nums:
        if len(np.intersect1d(hand, card_num)) == 4:
            return True
        else:
            return False

def isThreeOfAKind(hand):
    for card_num in card_nums:
        if len(np.intersect1d(hand, card_num)) == 3:
            return True
        else:
            return False

def isOnePair(hand):
    for card_num in card_nums:
        if len(np.intersect1d(hand, card_num)) == 2:
            return True
        else:
            return False

def isFullHouse(hand):
    if isThreeOfAKind(hand) and isOnePair(hand):
        return True
    else:
        return False

def isTwoPair(hand):
    pairs = 0
    for card_num in card_nums:
        if len(np.intersect1d(hand, card_num)) == 2:
            pairs += 1
    
    if pairs == 2:
        return True
    else:
        return False
   

In [62]:
def highCardTie(p1_hand, p2_hand):
    #convert list of card values to numbers/faces
    p1_nums = cardIntsToCardNums(p1_hand)
    p2_nums = cardIntsToCardNums(p2_hand)

    #compares each player's high card
    #if both hands have the same high card, then compares next highest cards
    #until there is a winner or all cards have been compared
    result = 2
    while(result == 2 and len(p1_nums) > 0):
        #find the highest card in each player's hand
        p1_high_card = np.max(p1_nums)
        p2_high_card = np.max(p2_nums)

        #compare each player's high card to determine winner
        if p1_high_card > p2_high_card:
            result = 0
        elif p1_high_card < p2_high_card:
            result = 1
        else:
            p1_nums.remove(p1_high_card)
            p2_nums.remove(p2_high_card)
    return result

def onePairTie(p1_hand, p2_hand):
    #find the pair each player has
    for card_num in card_nums:
        if len(np.intersect1d(p1_hand, card_num)) == 2:
            pair1 = np.intersect1d(p1_hand, card_num)
        if len(np.intersect1d(p2_hand, card_num)) == 2:
            pair2 = np.intersect1d(p2_hand, card_num)
    
    #find the number/face of each pair
    for i in range(len(pair1)):
        for j in range(len(card_nums)):
            if pair1[i] in card_nums[j]:
                pair1_type = j+2

    for i in range(len(pair2)):
        for j in range(len(card_nums)):
            if pair2[i] in card_nums[j]:
                pair2_type = j+2

    #compare pairs to determine the winner
    if pair1_type > pair2_type:
        result = 0
    elif pair1_type < pair2_type:
        result = 1
    else:
        result = highCardTie(p1_hand, p2_hand)  
    return result

def twoPairTie(p1_hand, p2_hand):
    #find the pairs and put both in a list for each player
    pairs_p1 = []
    pairs_p2 = []
    for card_num in card_nums:
        if len(np.intersect1d(p1_hand, card_num)) == 2:
            pairs_p1.append(np.intersect1d(p1_hand, card_num).tolist())
        if len(np.intersect1d(p2_hand, card_num)) == 2:
            pairs_p2.append(np.intersect1d(p2_hand, card_num).tolist())
    
    #order the pairs in each list from highest to lowest
    if onePairTie(pairs_p1[0], pairs_p1[1]) != 0:
        pairs_p1.reverse()
    
    if onePairTie(pairs_p2[0], pairs_p2[1]) != 0:
        pairs_p2.reverse()

    #compare pairs to determine winner
    tie_result = onePairTie(pairs_p1[0], pairs_p2[0])
    if tie_result == 0 or tie_result == 1:
        result = tie_result
    elif tie_result == 2:
        tie_result_2 = onePairTie(pairs_p1[1], pairs_p2[1])
        if tie_result_2 == 0 or tie_result_2 == 1:
            result = tie_result_2
        else:
            result = highCardTie(p1_hand, p2_hand)  
    return result

def threeOfAKindTie(p1_hand, p2_hand):
    #find the triple each player has
    for card_num in card_nums:
        if len(np.intersect1d(p1_hand, card_num)) == 3:
            triple1 = np.intersect1d(p1_hand, card_num)
        if len(np.intersect1d(p2_hand, card_num)) == 3:
            triple2 = np.intersect1d(p2_hand, card_num)

    #find the number/face of each triple
    for i in range(len(triple1)):
        for j in range(len(card_nums)):
            if triple1[i] in card_nums[j]:
                triple1_type = j+2

    for i in range(len(triple2)):
        for j in range(len(card_nums)):
            if triple2[i] in card_nums[j]:
                triple2_type = j+2
    
    #compare triples to determine the winner
    if triple1_type > triple2_type:
        result = 0
    elif triple1_type < triple2_type:
        result = 1
    else:
        result = highCardTie(p1_hand, p2_hand)
    return result

def straightTie(p1_hand, p2_hand):
    #can compare the highest card of each hand to compare 
    #stregnths of straights
    result = highCardTie(p1_hand, p2_hand)
    return result

def flushTie(p1_hand, p2_hand):
    #if two players have a flush, then the player with
    #the highest card wins
    result = highCardTie(p1_hand, p2_hand)
    return result

def fullHouseTie(p1_hand, p2_hand):
    #find the triple each player has
    for card_num in card_nums:
        if len(np.intersect1d(p1_hand, card_num)) == 3:
            triple1 = np.intersect1d(p1_hand, card_num)
        if len(np.intersect1d(p2_hand, card_num)) == 3:
            triple2 = np.intersect1d(p2_hand, card_num)

    #find the number/face of each triple
    for i in range(len(triple1)):
        for j in range(len(card_nums)):
            if triple1[i] in card_nums[j]:
                triple1_type = j+2

    for i in range(len(triple2)):
        for j in range(len(card_nums)):
            if triple2[i] in card_nums[j]:
                triple2_type = j+2
    
    #compare triples to determine the winner
    #if the triples are the same then
    #compare the pairs
    if triple1_type > triple2_type:
        result = 0
    elif triple1_type < triple2_type:
        result = 1
    else:
        result = onePairTie(p1_hand, p2_hand)
    return result

def fourOfAKindTie(p1_hand, p2_hand):
    #find the quartet each player has
    for card_num in card_nums:
        if len(np.intersect1d(p1_hand, card_num)) == 4:
            quartet1 = np.intersect1d(p1_hand, card_num)
        if len(np.intersect1d(p2_hand, card_num)) == 4:
            quartet2 = np.intersect1d(p2_hand, card_num)

    #find the number/face of each triple
    for i in range(len(quartet1)):
        for j in range(len(card_nums)):
            if quartet1[i] in card_nums[j]:
                quartet1_type = j+2

    for i in range(len(quartet2)):
        for j in range(len(card_nums)):
            if quartet2[i] in card_nums[j]:
                quartet2_type = j+2
    
    #compare quartets to determine the winner
    #if the quartets are the same then
    #compare the last card
    if quartet1_type > quartet2_type:
        result = 0
    elif quartet1_type < quartet2_type:
        result = 1
    else:
        result = highCardTie(p1_hand, p2_hand)
    return result

def straightFlushTie(p1_hand, p2_hand):
    #can compare the highest card of each hand to compare 
    #stregnths of straight flushes
    result = highCardTie(p1_hand, p2_hand)
    return result

In [116]:
#compare the hands of two players to see who wins between them
#returns 0 if player1 wins, 1 if player2 wins, and 2 if there's a tie
def compareTwoPlayers(player1_cards, player2_cards, dealer_cards):
    #make players' hands by combining their starting cards each with the dealer's cards
    player1_hand = np.concatenate((player1_cards, dealer_cards))
    player2_hand = np.concatenate((player2_cards, dealer_cards))

    #list to hold ranks of players' hands
    ranks = [10, 10]

    #determine rank of each player's hand
    #the ranks are as follows
    #Royal Flush: 1
    #Straight Flush: 2
    #Four of a Kind: 3
    #Full House: 4
    #Flush: 5
    #Straight: 6
    #Three of a Kind: 7
    #Two Pair: 8
    #One Pair: 9
    #High Card: 10
    hands = [player1_hand, player2_hand]
    for i in range(len(hands)):
        if isOnePair(hands[i]):
            ranks[i] = 9
        if isTwoPair(hands[i]):
            ranks[i] = 8
        if isThreeOfAKind(hands[i]):
            ranks[i] = 7
        if isStraight(hands[i]):
            ranks[i] = 6
        if isFlush(hands[i]):
            ranks[i] = 5
        if isFullHouse(hands[i]):
            ranks[i] = 4
        if isFourOfAKind(hands[i]):
            ranks[i] = 3
        if isStraightFlush(hands[i]):
            ranks[i] = 2
        if isRoyalFlush(hands[i]):
            ranks[i] = 1

    #compares the rank of each player's hand to determine the winner
    #if both hands have the same rank then compare strength
    #of each hand to determine the who is the winner or if game is a tie
    if ranks[0] < ranks[1]:
        result = 0
    elif ranks[0] > ranks[1]:
        result = 1
    else:
        if ranks[0] == 10:
            result = highCardTie(player1_hand, player2_hand)
        elif ranks[0] == 9:
            result = onePairTie(player1_hand, player2_hand)
        elif ranks[0] == 8:
            result = twoPairTie(player1_hand, player2_hand)
        elif ranks[0] == 7:
            result = threeOfAKindTie(player1_hand, player2_hand)
        elif ranks[0] == 6:
            result = straightTie(player1_hand, player2_hand)
        elif ranks[0] == 5:
            result = flushTie(player1_hand, player2_hand)
        elif ranks[0] == 4:
            result = fullHouseTie(player1_hand, player2_hand)
        elif ranks[0] == 3:
            result = fourOfAKindTie(player1_hand, player2_hand)
        elif ranks[0] == 2:
            result = straightFlushTie(player1_hand, player2_hand)
        elif ranks[0] == 1:
            result = 2

    return result

#compares your hand with all other players in the game
#to determine if you won, lost, or tied
#returns 0 if for win, 1 for loss, and 2 for a tie
def whoWin(start_cards, players_cards, dealer_cards):
    for i in range(len(players_cards)):
        result = compareTwoPlayers(start_cards,players_cards,dealer_cards)
    return result

In [117]:
#N - number of times to simulate the round
#n_players - number of other players in game
#starting_hand - your 2 starting cards you were dealt
def monteCarloPoker(N, n_players, starting_hand, starting_hand_opp):
    start = time.time()
    win_probability = 0
    lose_probability = 0
    tie_probability = 0
    print(starting_hand,starting_hand_opp)
    for i in range(N):
        #create deck
        deck = np.array([])
        for i in range(52):
            deck = np.append(deck, i)
        deck = deck.astype(int)

        #create starting hand
        start_hand = np.array([])
        for card in starting_hand:
            card_value = cardNameToInt(card)
            start_hand = np.append(start_hand, card_value)
        start_hand = start_hand.astype(int)
        
         #create opponent starting hand
        start_hand_opp = np.array([])
        for card in starting_hand_opp:
            card_value = cardNameToInt(card)
            start_hand_opp = np.append(start_hand_opp, card_value)
        start_hand_opp = start_hand_opp.astype(int)

        #remove starting_hand from deck
        index1 = np.argwhere(deck == start_hand[0])
        deck = np.delete(deck, index1)

        index2 = np.argwhere(deck == start_hand[1])
        deck = np.delete(deck, index2)
        
        #remove starting_hand_opp from deck
        index3 = np.argwhere(deck == start_hand_opp[0])
        deck = np.delete(deck, index3)

        index4 = np.argwhere(deck == start_hand_opp[1])
        deck = np.delete(deck, index4)

        #generate hands for game
        dealer_tup = generateDealerCards(deck)
        dealer_cards = dealer_tup[0]
        deck = dealer_tup[1]
    
        

        #determine winner
        result = whoWin(start_hand, start_hand_opp, dealer_cards)

        #determine probabilities
        if result == 0:
            win_probability += 1/N
        if result == 1:
            lose_probability += 1/N
        if result == 2:
            tie_probability += 1/N
    end = time.time()
    elapsed = end - start
    print(elapsed)
    return [win_probability, lose_probability, tie_probability]

In [118]:
N = 10000
card1 = "As"
card2 = "Ad"
starting_hand = [card1, card2]
card3 = "Ks"
card4 = "Kd"
starting_hand_opp = [card3, card4]
n_players = 2
results = monteCarloPoker(N, n_players, starting_hand, starting_hand_opp)
winPercent = results[0] * 100
losePercent = results[1] * 100
tiePercent = results[2] * 100
print("In {} different rounds, your hand of {} and {} won {:.3f}% of the rounds, lost {:.3f}% of the rounds, and tied {:.3f}% of the rounds.".format(N,card1,card2,winPercent,losePercent,tiePercent))



['As', 'Ad'] ['Ks', 'Kd']
54.59579682350159
In 10000 different rounds, your hand of As and Ad won 93.680% of the rounds, lost 6.320% of the rounds, and tied 0.000% of the rounds.
